In [ ]:
import os
import math
import glob
import cv2
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from moviepy.editor import VideoFileClip
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from scipy.ndimage.measurements import label
import pickle
%matplotlib inline

In [ ]:
test_images = np.array([plt.imread(i) for i in glob.glob('./test_images/*.jpg')])

car_images = []
non_car_images = []

for root, dirs, files in os.walk('./data/vehicles/'):
    for file in files:
        if file.endswith('.png'):
            car_images.append(os.path.join(root, file))
            
for root, dirs, files in os.walk('./data/non-vehicles/'):
    for file in files:
        if file.endswith('.png'):
            non_car_images.append(os.path.join(root, file))

In [ ]:
# Compute binned color features by scaling images down 
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


# Return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualize=vis, feature_vector=feature_vec)
        return features

# Extract feature wrapper that extracts and combines all features
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0, spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB' 
            
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)   

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features,hog_features)))
        
    # Return list of feature vectors
    return features

In [ ]:
def extract_features_single_image(img, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0, spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256),spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if cspace != 'RGB':
        if cspace == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
#         print(feature_image)
        
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
       
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)


In [ ]:
### Parameters
spatial = 32
hist_bins = 32
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb #YCrCb best
orient = 9
pix_per_cell = 8
cell_per_block = 2
spatial_size= (32, 32)
heat_threshold= 4 # 12
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL" #ALL,0 best
ystart_ystop_scale = [(405, 510, 1), (400, 600, 1.5), (500, 710, 2.5)]

color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
#orient = 12  # HOG orientations
#pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
#hog_channel = 0 # Can be 0, 1, 2, or "ALL"
#spatial_size = (16, 16) # Spatial binning dimensions
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()

car_features = extract_features(car_images, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_size=(spatial, spatial),
                        hist_bins=hist_bins, hist_range=(0, 256))

non_car_features = extract_features(non_car_images,cspace=colorspace,orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_size=(spatial, spatial),
                        hist_bins=hist_bins, hist_range=(0, 256))

In [ ]:
# Preserve State to compare parameter tuning 
rand_state = np.random.randint(0, 100)

# Create an array stack of feature vectors
X = np.vstack((car_features, non_car_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(non_car_features))))

# Split up data into randomized training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using spatial binning of:',spatial,'and', hist_bins,'histogram bins')
print('Feature vector length:', len(X_train[0]))

In [ ]:
# Use a linear SVC 
svc = LinearSVC()

# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()

print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

# Prediction 
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts:     ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5), offset=0):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = (img.shape[0]//2)+offset 
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]

    window_list = []

    
    scales=[1,1.5,2,2.5]
    ystart=y_start_stop[0]
    for scale in scales:
        over_x= xy_overlap[0]
        over_y= xy_overlap[1]
        width= xy_window[0]*scale
        height= xy_window[1]*scale
        c= y_start_stop[0]
        
        for row in range(2):
            col=0
            endx=0
            image_width= img.shape[1]
            
            while(endx<image_width):
                
                startx = x_start_stop[0]+ col*width*(1-over_x)
                endx = startx + width
                starty = ystart+ row*height*(1-over_y)
                endy = starty + height
                if(endx-startx==endy-starty and endx-startx!=0):
#                     print(endx-startx, endy-starty)
                    window_list.append(((int(startx), int(starty)), (int(endx), int(endy)), width/64.0))
                col+=1
#         y_start_stop[0]-=5    
    return window_list

In [ ]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    last_ratio=0
    i=0
    imageCopy= np.copy(img)
    for window in windows:
        if(last_ratio!= window[2]):
            last_ratio=window[2]
            imageCopy= cv2.resize(img, (int(img.shape[1]/last_ratio), int(img.shape[0]/last_ratio))) 
        #3) Extract the test window from original image
        test_img = imageCopy[int(window[0][1]/last_ratio):int(window[1][1]/last_ratio), int(window[0][0]/last_ratio):int(window[1][0]/last_ratio)]
        if test_img.shape[0]==test_img.shape[1] and test_img.shape[0]!=0:

        #4) Extract features for that window using single_img_features()
            features = extract_features_single_image(test_img, cspace=colorspace, orient=orient, 
                    pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                    hog_channel=hog_channel, spatial_size=(spatial, spatial),
                    hist_bins=hist_bins, hist_range=(0, 256))
            #5) Scale extracted features to be fed to classifier
    #         print(features)
            test_features = scaler.transform(np.array(features).reshape(1, -1))
    #         print(test_features)
            #6) Predict using your classifier
            prediction = clf.predict (test_features)

    #         #7) If positive (prediction == 1) then save the window
#             if prediction == 1:
            prob = clf.decision_function(test_features)
            window1=(window[0],window[1],prob)
            if(prob>1.5):
                on_windows.append(window1)
    #8) Return windows for positive detections
    return on_windows

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
#         print(bbox[2])
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        point=(bbox[0][0],bbox[0][1]-20)
        text= str(bbox[2])
        cv2.putText(imcopy,text,point, cv2.FONT_HERSHEY_SIMPLEX, 0.4,(255,0,0),1,cv2.LINE_AA)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
from scipy.ndimage.measurements import label

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    coord=[]
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        point=(np.min(nonzerox), np.min(nonzeroy)-20)
        text= 'Car '+ str(car_number)
        # Draw the box on the image
        aspect_ratio= float(bbox[1][0]-bbox[0][0])/(bbox[1][1]-bbox[0][1])
        if(aspect_ratio>=0.8 ):
            
            coord.append((bbox[0], bbox[1]))
    # Return the image
    return img, coord

def draw_objects(coords, draw_img):
    out_image = np.zeros((720,1280,3), dtype=np.uint8)
    index=1
    out_image[0:720,0:1280,:] = cv2.resize(draw_img,(1280,720)) 
    offset_left=30
    offset_start=30
    mid_gap=40
    
    for coord in coords:
        if(index<4):
            x_start= coord[0][0]
            y_start= coord[0][1]
            x_end= coord[1][0]
            y_end= coord[1][1]
            if x_end-x_start>70 and y_end-y_start>50:
                aspect_ratio= float(x_end-x_start)/(y_end-y_start)

                detected_image = np.zeros((150,220,3), dtype=np.uint8)
                if(aspect_ratio>=0.8 and aspect_ratio<=4):
                    detected_image= cv2.resize(draw_img[y_start+5:max(y_end-5,y_start+80),x_start+5:max(x_end-5,x_start+115)],(220,150))
                    cv2.rectangle(out_image,(offset_left-2,offset_start-2),(offset_left+222,offset_start+152 ), (255,255,255), 3)
                else:
                    cv2.putText(detected_image,'Bad Quality ',(50,80), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,0,0),2,cv2.LINE_AA)
                    cv2.rectangle(out_image,(offset_left-2,offset_start-2),(offset_left+222,offset_start+152 ), (255,0,0), 3)
                cv2.rectangle(out_image, coord[0], coord[1], (226,203,255), 6)
                out_image[offset_start:offset_start+150,offset_left:offset_left+220,:] = cv2.resize(detected_image,(220,150))
                cv2.putText(out_image,'Car '+str(index)+' detected',(offset_left,offset_start-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(255,255,255),2,cv2.LINE_AA)
                offset_start+=150+mid_gap
        index+=1
    return out_image

In [ ]:
prev_heat=[]

def avg_heat(heat, span):
    global prev_heat
    
    if(len(prev_heat)>span):
        prev_heat.pop(0)
    prev_heat.append(heat)
    return np.mean(prev_heat, axis=0)

prev_windows_left=[]
prev_windows_right=[]
def averaging(curr_windows, span, window_side):
    
    global prev_windows_left
    global prev_windows_right
    result=[]
    if window_side=='R':
        no_of_window_slots=len(prev_windows_right)
        if(no_of_window_slots>span):
            prev_windows_right.pop(0)
        prev_windows_right.append(curr_windows)
        
        for windows in prev_windows_right:

            for window in windows:
                result.append(window)
    else:   
        no_of_window_slots=len(prev_windows_left)
        if(no_of_window_slots>span):
            prev_windows_left.pop(0)
        prev_windows_left.append(curr_windows)
        
        for windows in prev_windows_left:

            for window in windows:
                result.append(window)
        
    return result,no_of_window_slots
    
  

In [ ]:
prev_windows_left=[]
prev_windows_right=[]
prev_heat=[]
windows_final = slide_window(cv2.imread("./test8.jpg"), x_start_stop=[600, None], y_start_stop=[None, None], 
                       xy_window=(72, 72), xy_overlap=(0.9,0.9), offset=40)

last_frame=None
frame_no=0
last_left=None
last_right=None
def Object_pipeline(img):
    global windows_final
    global svc
    global X_scaler
    global color_space
    global spatial_size
    global hist_bins
    global orient
    global pix_per_cell
    global cell_per_block
    global hog_channel
    global spatial_feat 
    global hist_feat 
    global hog_feat 
    global frame_no
    global last_frame
    global last_left
    global last_right
    global prev_heat
   
    result=None


    

    hot_windows = search_windows(img, windows_final, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat = add_heat(heat,hot_windows)
    heat = avg_heat(heat, 20)
    heat = apply_threshold(heat,max(np.max(heat)//2-15 ,2))
    heatmap = np.clip(heat, 0, 255)
    labels = label(heatmap)
    draw_img, coord = draw_labeled_bboxes(np.copy(img), labels)
    
    draw_img= draw_objects(coord, draw_img)
    last_frame=draw_img
    result= draw_img
    frame_no+=1
    return result


In [17]:
prev_windows_left=[]
prev_windows_right=[]
prev_heat=[]
frame_no=0
last_frame=None
from moviepy.editor import VideoFileClip

video_output1 = 'project_video_output.mp4'
video_input1 = VideoFileClip('project_video.mp4')
processed_video = video_input1.fl_image(Object_pipeline)
%time processed_video.write_videofile(video_output1, audio=False)
video_input1.reader.close()
video_input1.audio.reader.close_proc()

Moviepy - Building video project_video_output.mp4.
Moviepy - Writing video project_video_output.mp4



t:  13%|█▎        | 159/1260 [07:19<58:20,  3.18s/it, now=None] 